### **Experiment Log:**

|Version |Models Used |CV Score |LB Score| Changes Made
| --- | --- | --- | --- | --- |
|v1 | LightGBM, CatBoost <br> Deep Neural Network | 0.19779 | 0.20699 | Baseline
|v2 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.19171 | 0.20484 | Linear regression (meta model) added
|v3 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.19106 | 0.20482 | New features added
|v4 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.19099 | 0.20449 | K-Fold stratification method modified
|v5 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.19065 | 0.20549 | New lag features added
|v6 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18927 | Error | New lag features added <br> Models architecture modified
|v7 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18941 | 0.20243 | Modified feature engineering process
|v8 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.189193 | 0.20227 | Models architecture modified
|v9 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18919 | 0.20261 | DNN model architecture modified
|v10 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18915 | NA | DNN model architecture modified
|v11 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18922 | 0.20222 | DNN model architecture modified
|v12 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18873 | 0.20259 | DNN model architecture modified
|v13 | LightGBM, CatBoost <br> Deep Neural Network | 0.19200 | 0.20259 | Weighted average ensemble
|v14 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18888 | Error | New features added
|v15 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18764 | 0.20332 | Quantile transformation replaced with Min Max Scaler
|v16 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18768 | Error | Missing values replaced with mean
|v17 | LightGBM, CatBoost <br> Deep Neural Network <br> Gradient Boosting Regressor <br> Linear Regression (Meta) | 0.20154 | 0.20485 | Modified feature engineering process <br> New base model added
|v18 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | 0.18807 | 0.20660 | New historical volatility features added
|v23 | LightGBM, CatBoost <br> Deep Neural Network <br> Linear Regression (Meta) | TBD | TBD | Kfold cross validation <br> Special handling for DNN model

## Import libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import gc
import glob
import random
import pickle
import joblib
import numpy.matlib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import LeakyReLU, Reshape, Add
from tensorflow.keras.layers import Activation, Concatenate
from tensorflow.keras.layers import SpatialDropout1D, Conv1D
from tensorflow.keras.layers import AveragePooling1D, Dropout
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.layers import Input, BatchNormalization

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(2021)

## Helper Functions

In [ ]:
def rmspe(y_true, y_pred):
    return (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

In [ ]:
def rmspe_loss(y_true, y_pred):
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    return (tf.math.sqrt(tf.reduce_mean(tf.math.square((y_true - y_pred) / y_true))))

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
def historical_volatility(series):
    return np.sqrt(np.sum(series**2)/len(series))

In [ ]:
def count_unique(series):
    return len(np.unique(series))

In [ ]:
def get_stats_window(df, fe_dict, seconds_in_bucket, add_suffix = False):
    df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]
    
    if add_suffix:
        df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
    
    return df_feature

In [ ]:
def process_trade_data(trade_files):
    
    trade_df = pd.DataFrame()
    
    for file in tqdm(glob.glob(trade_files)):
        
        # Read source file
        df_trade_data = pd.read_parquet(file)
        
        # Feature engineering
        df_trade_data['log_return'] = df_trade_data.groupby('time_id')['price'].apply(log_return)
        df_trade_data['size_order_count_ratio'] = df_trade_data.apply(lambda x: 0 if x['order_count'] == 0 else x['size']/x['order_count'], axis=1)
        price_mean = df_trade_data['price'].mean()
        df_trade_data['price1'] = df_trade_data['price'].apply(lambda x: x - price_mean)
        
        fet_engg_dict = {
            'price': ['mean','std','sum'],
            'size': ['mean','std','sum'],
            'order_count': ['mean','std','sum'],
            'size_order_count_ratio': ['mean','std','sum'],
            'seconds_in_bucket': [count_unique],
            'log_return': [realized_volatility,'mean','std','sum'],
            'price1': [historical_volatility,'mean','std','sum']
        }
        
        # Get the stats for different windows
        df_feature = get_stats_window(df_trade_data, fet_engg_dict, seconds_in_bucket = 0, add_suffix = False)
        df_feature_120 = get_stats_window(df_trade_data, fet_engg_dict, seconds_in_bucket = 120, add_suffix = True)
        df_feature_240 = get_stats_window(df_trade_data, fet_engg_dict, seconds_in_bucket = 240, add_suffix = True)
        df_feature_360 = get_stats_window(df_trade_data, fet_engg_dict, seconds_in_bucket = 360, add_suffix = True)
        df_feature_480 = get_stats_window(df_trade_data, fet_engg_dict, seconds_in_bucket = 480, add_suffix = True)
        
        # Merge all
        trade_agg_df = df_feature.merge(df_feature_120, how = 'left', left_on = 'time_id_', right_on = 'time_id__120')
        trade_agg_df = trade_agg_df.merge(df_feature_240, how = 'left', left_on = 'time_id_', right_on = 'time_id__240')
        trade_agg_df = trade_agg_df.merge(df_feature_360, how = 'left', left_on = 'time_id_', right_on = 'time_id__360')
        trade_agg_df = trade_agg_df.merge(df_feature_480, how = 'left', left_on = 'time_id_', right_on = 'time_id__480')
        trade_agg_df = trade_agg_df.add_prefix('trade_')
        
        # Generate row_id
        stock_id = file.split('=')[1]
        trade_agg_df['row_id'] = trade_agg_df['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
        trade_agg_df.drop(['trade_time_id_'], inplace=True, axis=1)
        
        # Merge with parent df
        trade_df = pd.concat([trade_df, trade_agg_df])
    
    del df_trade_data, trade_agg_df, df_feature
    del df_feature_120, df_feature_240
    del df_feature_360, df_feature_480
    gc.collect()
    
    return trade_df

In [ ]:
def process_book_data(book_files):
    
    book_df = pd.DataFrame()
    
    for file in tqdm(glob.glob(book_files)):
        
        # Read source file
        df_book_data = pd.read_parquet(file)
        
        # Feature engineering
        df_book_data['wap1'] = (df_book_data['bid_price1'] *
                                df_book_data['ask_size1'] +
                                df_book_data['ask_price1'] *
                                df_book_data['bid_size1'])  / (df_book_data['bid_size1'] +
                                                               df_book_data['ask_size1'])

        df_book_data['wap2'] = (df_book_data['bid_price2'] *
                                df_book_data['ask_size2'] +
                                df_book_data['ask_price2'] *
                                df_book_data['bid_size2'])  / (df_book_data['bid_size2'] +
                                                               df_book_data['ask_size2'])
        
        df_book_data['wap3'] = (df_book_data['bid_price1'] *
                                df_book_data['bid_size1'] +
                                df_book_data['ask_price1'] *
                                df_book_data['ask_size1'])  / (df_book_data['bid_size1'] +
                                                               df_book_data['ask_size1'])

        df_book_data['wap4'] = (df_book_data['bid_price2'] *
                                df_book_data['bid_size2'] +
                                df_book_data['ask_price2'] *
                                df_book_data['ask_size2'])  / (df_book_data['bid_size2'] +
                                                               df_book_data['ask_size2'])

        df_book_data['log_return1'] = df_book_data.groupby(['time_id'])['wap1'].apply(log_return)
        df_book_data['log_return2'] = df_book_data.groupby(['time_id'])['wap2'].apply(log_return)
        df_book_data['log_return3'] = df_book_data.groupby(['time_id'])['wap3'].apply(log_return)
        df_book_data['log_return4'] = df_book_data.groupby(['time_id'])['wap4'].apply(log_return)
        
        wap1_mean = df_book_data['wap1'].mean()
        df_book_data['waph1'] = df_book_data['wap1'].apply(lambda x: x - wap1_mean)
        
        wap2_mean = df_book_data['wap2'].mean()
        df_book_data['waph2'] = df_book_data['wap2'].apply(lambda x: x - wap2_mean)
        
        wap3_mean = df_book_data['wap3'].mean()
        df_book_data['waph3'] = df_book_data['wap3'].apply(lambda x: x - wap3_mean)
        
        wap4_mean = df_book_data['wap4'].mean()
        df_book_data['waph4'] = df_book_data['wap4'].apply(lambda x: x - wap4_mean)
        
        df_book_data['wap_balance'] = abs(df_book_data['wap1'] - df_book_data['wap2'])
        df_book_data['price_spread1'] = (df_book_data['ask_price1'] - df_book_data['bid_price1']) / ((df_book_data['ask_price1'] + df_book_data['bid_price1'])/2)
        df_book_data['price_spread2'] = (df_book_data['ask_price2'] - df_book_data['bid_price2']) / ((df_book_data['ask_price2'] + df_book_data['bid_price2'])/2)
        df_book_data['bid_spread'] = df_book_data['bid_price1'] - df_book_data['bid_price2']
        df_book_data['ask_spread'] = df_book_data['ask_price1'] - df_book_data['ask_price2']
        df_book_data['bid_ask_spread1'] = abs((df_book_data['bid_price1'] * df_book_data['bid_size1']) - (df_book_data['ask_price1'] * df_book_data['ask_size1']))
        df_book_data['bid_ask_spread2'] = abs((df_book_data['bid_price2'] * df_book_data['bid_size2']) - (df_book_data['ask_price2'] * df_book_data['ask_size2']))
        df_book_data['total_volume'] = (df_book_data['ask_size1'] + df_book_data['ask_size2']) + (df_book_data['bid_size1'] + df_book_data['bid_size2'])
        df_book_data['volume_imbalance'] = abs((df_book_data['ask_size1'] + df_book_data['ask_size2']) - (df_book_data['bid_size1'] + df_book_data['bid_size2']))
        
        fet_engg_dict = {
            'wap1': ['mean','std','sum'],
            'log_return1': [realized_volatility,'mean','std','sum'],
            'log_return2': [realized_volatility,'mean','std','sum'],
            'log_return3': [realized_volatility,'mean','std','sum'],
            'log_return4': [realized_volatility,'mean','std','sum'],
            'wap_balance': ['mean','std','sum'],
            'price_spread1': ['mean','std','sum'],
            'price_spread2': ['mean','std','sum'],
            'bid_spread': ['mean','std','sum'],
            'ask_spread': ['mean','std','sum'],
            'bid_ask_spread1': ['mean','std','sum'],
            'bid_ask_spread2': ['mean','std','sum'],
            'total_volume': ['mean','std','sum'],
            'volume_imbalance': ['mean','std','sum'],
            'waph1': [historical_volatility,'mean','std','sum']
        }
        
        fet_engg_time_dict = {
            'log_return1': [realized_volatility,'mean','std','sum'],
            'log_return2': [realized_volatility,'mean','std','sum'],
            'log_return3': [realized_volatility,'mean','std','sum'],
            'log_return4': [realized_volatility,'mean','std','sum'],
            'waph1': [historical_volatility,'mean','std','sum']
        }
        
        # Get the stats for different windows
        df_feature = get_stats_window(df_book_data, fet_engg_dict, seconds_in_bucket = 0, add_suffix = False)
        df_feature_120 = get_stats_window(df_book_data, fet_engg_time_dict, seconds_in_bucket = 120, add_suffix = True)
        df_feature_240 = get_stats_window(df_book_data, fet_engg_time_dict, seconds_in_bucket = 240, add_suffix = True)
        df_feature_360 = get_stats_window(df_book_data, fet_engg_time_dict, seconds_in_bucket = 360, add_suffix = True)
        df_feature_480 = get_stats_window(df_book_data, fet_engg_time_dict, seconds_in_bucket = 480, add_suffix = True)

        # Merge all
        book_agg_df = df_feature.merge(df_feature_120, how = 'left', left_on = 'time_id_', right_on = 'time_id__120')
        book_agg_df = book_agg_df.merge(df_feature_240, how = 'left', left_on = 'time_id_', right_on = 'time_id__240')
        book_agg_df = book_agg_df.merge(df_feature_360, how = 'left', left_on = 'time_id_', right_on = 'time_id__360')
        book_agg_df = book_agg_df.merge(df_feature_480, how = 'left', left_on = 'time_id_', right_on = 'time_id__480')
        book_agg_df = book_agg_df.add_prefix('book_')
        
        # Generate row_id
        stock_id = file.split('=')[1]
        book_agg_df['row_id'] = book_agg_df['book_time_id_'].apply(lambda x:f'{stock_id}-{x}')
        book_agg_df.drop(['book_time_id_'], inplace=True, axis=1)
        
        # Merge with parent df
        book_df = pd.concat([book_df, book_agg_df])
    
    del df_book_data, book_agg_df, df_feature
    del df_feature_120, df_feature_240
    del df_feature_360, df_feature_480
    gc.collect()
    
    return book_df

## Load training data

In [ ]:
with open("../input/orvp-django-unchained/ORVP_Ready_Meatballs.txt", 'rb') as handle: 
    data = handle.read()

processed_data = pickle.loads(data)
train_df = processed_data['train_df']

del processed_data
gc.collect()

## Prepare testing data

### Test data

In [ ]:
test_df = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
test_df['row_id'] = test_df['stock_id'].astype(str) + '-' + test_df['time_id'].astype(str)
print(f"test_df: {test_df.shape}")
test_df.head()

### Trade data

In [ ]:
trade_test_df = process_trade_data('../input/optiver-realized-volatility-prediction/trade_test.parquet/*')
print(f"trade_test_df: {trade_test_df.shape}")
trade_test_df.head()

In [ ]:
test_df = pd.merge(test_df, trade_test_df, 
                   how='left', on='row_id', 
                   sort=False)

print(f"test_df: {test_df.shape}")
test_df.head()

### Book data

In [ ]:
book_test_df = process_book_data('../input/optiver-realized-volatility-prediction/book_test.parquet/*')
print(f"book_test_df: {book_test_df.shape}")
book_test_df.head()

In [ ]:
test_df = pd.merge(test_df, book_test_df, 
                   how='left', on='row_id', 
                   sort=False)

print(f"test_df: {test_df.shape}")
test_df.head()

### Group features

In [ ]:
vol_cols = []
for col in test_df.columns:
    if 'realized_volatility' in col:
        vol_cols.append(col)

len(vol_cols)

In [ ]:
df_stock_id = test_df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'sum']).reset_index()
df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
df_stock_id = df_stock_id.add_suffix('_stock')
df_stock_id.head()

In [ ]:
df_time_id = test_df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', 'sum']).reset_index()
df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
df_time_id = df_time_id.add_suffix('_time')
df_time_id.head()

In [ ]:
test_df = test_df.merge(df_stock_id, how='left', 
                          left_on=['stock_id'], 
                          right_on=['stock_id__stock'])

test_df = test_df.merge(df_time_id, how='left', 
                          left_on=['time_id'], 
                          right_on=['time_id__time'])

test_df.drop(['stock_id__stock', 'time_id__time'], 
              axis = 1, inplace = True)

del df_stock_id, df_time_id
gc.collect()

print(f"test_df: {test_df.shape}")

In [ ]:
test_df['size_tau'] = np.sqrt(1/test_df['trade_seconds_in_bucket_count_unique'])
test_df['size_tau_120'] = np.sqrt(1/test_df['trade_seconds_in_bucket_count_unique_120'])
test_df['size_tau_240'] = np.sqrt(1/test_df['trade_seconds_in_bucket_count_unique_240'])
test_df['size_tau_360'] = np.sqrt(1/test_df['trade_seconds_in_bucket_count_unique_360'])
test_df['size_tau_480'] = np.sqrt(1/test_df['trade_seconds_in_bucket_count_unique_480'])

test_df['size_tau2'] = np.sqrt(1/test_df['trade_order_count_sum'])
test_df['size_tau2_120'] = np.sqrt(0.8/test_df['trade_order_count_sum'])
test_df['size_tau2_240'] = np.sqrt(0.6/test_df['trade_order_count_sum'])
test_df['size_tau2_360'] = np.sqrt(0.4/test_df['trade_order_count_sum'])
test_df['size_tau2_480'] = np.sqrt(0.2/test_df['trade_order_count_sum'])

print(f"test_df: {test_df.shape}")
test_df.head()

In [ ]:
filter_cols = ['trade_time_id__120', 'trade_time_id__240', 'trade_time_id__360', 'trade_time_id__480', 
               'book_time_id__120', 'book_time_id__240', 'book_time_id__360', 'book_time_id__480']

test_df.drop(filter_cols, axis=1, inplace=True)
print(f"test_df: {test_df.shape}")
test_df.head()

## Additional features

In [ ]:
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()
ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)

l = []
for n in range(7):
    l.append([(x-1) for x in ((ids+1)*(kmeans.labels_ == n)) if x > 0])

mat = []
matTest = []

n = 0
for ind in tqdm(l):
    newDf = train_df.loc[train_df['stock_id'].isin(ind)]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append(newDf)
    
    newDf = test_df.loc[test_df['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append(newDf)
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)
mat2 = pd.concat(matTest).reset_index()

mat2 = pd.concat([mat2, mat1.loc[mat1.time_id==5]])
mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

In [ ]:
nnn = [
    'time_id',
    'trade_size_sum_0c1',
    'trade_size_sum_1c1',
    'trade_size_sum_3c1',
    'trade_size_sum_4c1',
    'trade_size_sum_6c1',
    'trade_order_count_sum_0c1',
    'trade_order_count_sum_1c1',
    'trade_order_count_sum_2c1',
    'trade_order_count_sum_4c1',
    'trade_order_count_sum_6c1',
    'trade_log_return_realized_volatility_0c1',
    'trade_log_return_realized_volatility_1c1',
    'trade_log_return_realized_volatility_3c1',
    'trade_log_return_realized_volatility_4c1',
    'trade_log_return_realized_volatility_6c1',
    'book_log_return1_realized_volatility_0c1',
    'book_log_return1_realized_volatility_1c1',
    'book_log_return1_realized_volatility_3c1',
    'book_log_return1_realized_volatility_4c1',
    'book_log_return1_realized_volatility_6c1',
    'book_bid_spread_sum_0c1',
    'book_bid_spread_sum_1c1',
    'book_bid_spread_sum_3c1',
    'book_bid_spread_sum_4c1',
    'book_bid_spread_sum_6c1',
    'book_ask_spread_sum_0c1',
    'book_ask_spread_sum_1c1',
    'book_ask_spread_sum_3c1',
    'book_ask_spread_sum_4c1',
    'book_ask_spread_sum_6c1',
    'book_total_volume_sum_0c1',
    'book_total_volume_sum_1c1',
    'book_total_volume_sum_3c1',
    'book_total_volume_sum_4c1',
    'book_total_volume_sum_6c1',
    'book_volume_imbalance_sum_0c1',
    'book_volume_imbalance_sum_1c1',
    'book_volume_imbalance_sum_3c1',
    'book_volume_imbalance_sum_4c1',
    'book_volume_imbalance_sum_6c1',
    'book_waph1_historical_volatility_0c1',
    'book_waph1_historical_volatility_1c1',
    'book_waph1_historical_volatility_3c1',
    'book_waph1_historical_volatility_4c1',
    'book_waph1_historical_volatility_6c1',
    'size_tau_0c1',
    'size_tau_1c1',
    'size_tau_3c1',
    'size_tau_4c1',
    'size_tau_6c1',
]

train_df = pd.merge(train_df, mat1[nnn], how='left', on='time_id')
test_df = pd.merge(test_df, mat2[nnn], how='left', on='time_id')

del mat1, mat2
gc.collect()

train_df.shape, test_df.shape

In [ ]:
colNames = [col for col in list(train_df.columns) if col not in {"stock_id", "time_id", "target", "row_id"}]
train_nn = train_df[colNames].copy()
test_nn = test_df[colNames].copy()

train_nn.replace([np.inf, -np.inf], np.nan, inplace=True)
test_nn.replace([np.inf, -np.inf], np.nan, inplace=True)

## Correlation check

In [ ]:
# Get highly correlated columns to remove
# df = train_df.loc[:, train_df.columns != 'target'].copy()
# cor_matrix = df.corr().abs()
# upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
# print(f"Number of columns to drop: {len(to_drop)}")
# select_cols = [col for col in test_df.columns if col not in to_drop]
# print(f"Selected columns: {len(select_cols)}")
# print(select_cols)

In [ ]:
select_cols = [
    'stock_id', 'time_id', 'row_id', 'trade_price_mean', 'trade_price_std', 'trade_price_sum', 'trade_size_mean', 'trade_size_std', 'trade_size_sum', 
    'trade_order_count_mean', 'trade_order_count_std', 'trade_order_count_sum', 'trade_size_order_count_ratio_mean', 'trade_size_order_count_ratio_std',
    'trade_size_order_count_ratio_sum', 'trade_log_return_realized_volatility', 'trade_log_return_mean', 'trade_log_return_std', 'trade_log_return_sum',
    'trade_price1_historical_volatility', 'trade_price1_sum', 'trade_order_count_std_120', 'trade_log_return_mean_120', 'trade_log_return_sum_120',
    'trade_price_std_240', 'trade_size_mean_240', 'trade_order_count_std_240', 'trade_log_return_mean_240', 'trade_log_return_sum_240', 'trade_price_std_360',
    'trade_size_mean_360', 'trade_size_std_360', 'trade_order_count_mean_360', 'trade_order_count_std_360', 'trade_size_order_count_ratio_std_360',
    'trade_log_return_mean_360', 'trade_log_return_sum_360', 'trade_price_std_480', 'trade_size_mean_480', 'trade_size_std_480', 'trade_size_sum_480',
    'trade_order_count_mean_480', 'trade_order_count_std_480', 'trade_size_order_count_ratio_mean_480', 'trade_size_order_count_ratio_std_480',
    'trade_log_return_realized_volatility_480', 'trade_log_return_mean_480', 'trade_log_return_std_480', 'trade_log_return_sum_480', 'book_wap1_sum',
    'book_log_return1_realized_volatility', 'book_log_return1_mean', 'book_log_return3_mean', 'book_log_return4_mean', 'book_wap_balance_mean',
    'book_wap_balance_sum', 'book_price_spread1_std', 'book_price_spread2_sum', 'book_bid_spread_mean', 'book_bid_spread_std', 'book_bid_spread_sum',
    'book_ask_spread_mean', 'book_ask_spread_std', 'book_ask_spread_sum', 'book_bid_ask_spread1_mean', 'book_bid_ask_spread1_std', 'book_bid_ask_spread2_mean',
    'book_bid_ask_spread2_std', 'book_total_volume_mean', 'book_total_volume_std', 'book_volume_imbalance_mean', 'book_log_return1_mean_120',
    'book_log_return2_mean_120', 'book_log_return3_mean_120', 'book_log_return4_mean_120', 'book_log_return1_mean_240', 'book_log_return2_mean_240',
    'book_log_return3_mean_240', 'book_log_return4_mean_240', 'book_log_return1_mean_360', 'book_log_return2_mean_360', 'book_log_return3_mean_360',
    'book_log_return4_mean_360', 'book_log_return1_mean_480', 'book_log_return1_sum_480', 'book_log_return2_mean_480', 'book_log_return2_sum_480', 
    'book_log_return3_mean_480', 'book_log_return3_sum_480', 'book_log_return4_mean_480', 'book_log_return4_sum_480', 'book_waph1_std_480',
    'trade_log_return_realized_volatility_mean_stock', 'trade_log_return_realized_volatility_std_stock', 'trade_log_return_realized_volatility_max_stock',
    'trade_log_return_realized_volatility_min_stock', 'trade_log_return_realized_volatility_120_max_stock', 'trade_log_return_realized_volatility_120_min_stock',
    'trade_log_return_realized_volatility_240_max_stock', 'trade_log_return_realized_volatility_240_min_stock', 
    'trade_log_return_realized_volatility_360_max_stock', 'trade_log_return_realized_volatility_360_min_stock', 
    'trade_log_return_realized_volatility_480_max_stock', 'trade_log_return_realized_volatility_480_min_stock', 
    'book_log_return1_realized_volatility_mean_stock', 'book_log_return1_realized_volatility_std_stock', 'book_log_return1_realized_volatility_max_stock',
    'book_log_return1_realized_volatility_min_stock', 'book_log_return2_realized_volatility_max_stock', 'book_log_return2_realized_volatility_min_stock',
    'book_log_return3_realized_volatility_max_stock', 'book_log_return3_realized_volatility_min_stock', 'book_log_return4_realized_volatility_max_stock',
    'book_log_return4_realized_volatility_min_stock', 'book_log_return1_realized_volatility_120_max_stock', 'book_log_return2_realized_volatility_120_min_stock',
    'book_log_return1_realized_volatility_240_min_stock', 'book_log_return2_realized_volatility_240_min_stock', 
    'book_log_return3_realized_volatility_240_min_stock', 'book_log_return1_realized_volatility_360_min_stock', 
    'book_log_return2_realized_volatility_360_min_stock', 'book_log_return3_realized_volatility_360_min_stock', 
    'book_log_return4_realized_volatility_360_min_stock', 'book_log_return1_realized_volatility_480_max_stock', 
    'book_log_return1_realized_volatility_480_min_stock', 'book_log_return2_realized_volatility_480_max_stock', 
    'book_log_return2_realized_volatility_480_min_stock', 'book_log_return3_realized_volatility_480_max_stock', 
    'book_log_return3_realized_volatility_480_min_stock', 'book_log_return4_realized_volatility_480_max_stock', 
    'book_log_return4_realized_volatility_480_min_stock', 'trade_log_return_realized_volatility_mean_time', 'trade_log_return_realized_volatility_std_time',
    'trade_log_return_realized_volatility_max_time', 'trade_log_return_realized_volatility_min_time', 'trade_log_return_realized_volatility_360_min_time',
    'trade_log_return_realized_volatility_480_max_time', 'trade_log_return_realized_volatility_480_min_time', 'book_log_return1_realized_volatility_std_time',
    'book_log_return1_realized_volatility_min_time', 'book_log_return2_realized_volatility_480_min_time', 'size_tau', 'size_tau_480', 'size_tau2',
    'trade_size_sum_0c1', 'trade_size_sum_1c1', 'trade_size_sum_3c1', 'trade_size_sum_4c1', 'trade_size_sum_6c1', 'trade_order_count_sum_1c1',
    'trade_order_count_sum_2c1', 'trade_order_count_sum_4c1', 'trade_order_count_sum_6c1', 'trade_log_return_realized_volatility_1c1', 'book_bid_spread_sum_1c1',
    'book_ask_spread_sum_1c1', 'book_total_volume_sum_0c1', 'book_total_volume_sum_1c1', 'book_total_volume_sum_3c1', 'book_total_volume_sum_4c1',
    'book_total_volume_sum_6c1', 'book_volume_imbalance_sum_0c1', 'book_volume_imbalance_sum_3c1', 'book_volume_imbalance_sum_4c1',
    'book_volume_imbalance_sum_6c1', 'book_waph1_historical_volatility_0c1', 'book_waph1_historical_volatility_1c1', 'book_waph1_historical_volatility_3c1',
    'book_waph1_historical_volatility_4c1', 'size_tau_0c1', 'size_tau_1c1', 'size_tau_3c1', 'size_tau_4c1', 'size_tau_6c1', 'target'
]

len(select_cols)

In [ ]:
train_df = train_df[select_cols].copy()
test_df = test_df[[col for col in select_cols if col!='target']].copy()
print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")

## LightGBM

In [ ]:
FOLD = 5
SEEDS = [2018]

fet_imp = 0
counter = 0
oof_score = 0
y_pred_meta_lgb = np.zeros((train_df.shape[0], 1))
y_pred_final_lgb = 0

cat_cols_indices = [train_df.columns.get_loc('stock_id')]
features = [col for col in train_df.columns if col not in {"time_id", "target", "row_id"}]
Xtrain = train_df[features].copy()
Ytrain = train_df['target'].copy()
Xtest = test_df[features].copy()

for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = KFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df)):
        counter += 1
        
        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]
        sample_weight = 1/np.square(train_y)
        
        model = LGBMRegressor(
            boosting_type='gbdt', 
            num_leaves=175, 
            max_depth=9, 
            learning_rate=0.05, 
            n_estimators=1500,
            objective='regression', 
            importance_type='gain',
            device='gpu',
            max_bin=100, 
            min_child_samples=500, 
            subsample=0.85, 
            subsample_freq=5, 
            colsample_bytree=0.55, 
            reg_lambda=0.05,
            random_state=0,
            bagging_seed=0,
            feature_fraction_seed=0
        )
        
        model.fit(train_x, train_y, eval_metric='rmse',
                  eval_set=[(train_x, train_y), (val_x, val_y)],
                  early_stopping_rounds=100, verbose=500,
                  categorical_feature=cat_cols_indices,
                  sample_weight=sample_weight)

        y_pred = model.predict(val_x, num_iteration=model.best_iteration_)
        y_pred_meta_lgb[val] += np.array([y_pred]).T
        y_pred_final_lgb += model.predict(Xtest, num_iteration=model.best_iteration_)
        
        score = rmspe(val_y, y_pred)
        oof_score += score
        seed_score += score
        fet_imp += model.feature_importances_
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        joblib.dump(model, f'./lgb_model_{idx + 1}C.txt')
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_lgb = y_pred_meta_lgb / float(len(SEEDS))
y_pred_final_lgb = y_pred_final_lgb / float(counter)
fet_imp = fet_imp / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

del Xtrain, Ytrain, Xtest
gc.collect()

## CatBoost

In [ ]:
FOLD = 5
SEEDS = [2018]

fet_imp = 0
counter = 0
oof_score = 0
y_pred_meta_cb = np.zeros((train_df.shape[0], 1))
y_pred_final_cb = 0

cat_cols_indices = [train_df.columns.get_loc('stock_id')]
features = [col for col in train_df.columns if col not in {"time_id", "target", "row_id"}]
Xtrain = train_df[features].copy()
Ytrain = train_df['target'].copy()
Xtest = test_df[features].copy()


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = KFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_df)):
        counter += 1
        
        train_x, train_y = Xtrain.iloc[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain.iloc[val], Ytrain.iloc[val]
        sample_weight = 1/np.square(train_y)
        
        model = CatBoostRegressor(
            objective='RMSE',
            eval_metric='RMSE',
            num_boost_round=1500,
            max_ctr_complexity=15,
            od_wait=500, 
            od_type='Iter',
            learning_rate=0.05,
            reg_lambda=0.05,
            bootstrap_type='Poisson',
            use_best_model=True, 
            subsample=0.8,
            grow_policy='Lossguide',
            min_data_in_leaf=500, 
            task_type='GPU',
            random_state=0
        )

        model.fit(train_x, train_y, eval_set=[(val_x, val_y)], 
                  cat_features=cat_cols_indices, 
                  sample_weight=sample_weight, 
                  early_stopping_rounds=100, verbose=500)

        y_pred = model.predict(val_x)
        y_pred_meta_cb[val] += np.array([y_pred]).T
        y_pred_final_cb += model.predict(Xtest)
        
        score = rmspe(val_y, y_pred)
        oof_score += score
        seed_score += score
        fet_imp += model.feature_importances_
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        joblib.dump(model, f'./cb_model_{idx + 1}C.txt')
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_cb = y_pred_meta_cb / float(len(SEEDS))
y_pred_final_cb = y_pred_final_cb / float(counter)
fet_imp = fet_imp / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

del Xtrain, Ytrain, Xtest
gc.collect()

## Deep Neural Model

In [ ]:
# Generate kfolds based on the knn++ algorithm
out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')
out_train = out_train.fillna(out_train.mean())

nfolds = 5
index = []
totDist = []
values = []

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = out_train.values
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds)

mat = np.c_[mat, np.arange(mat.shape[0])]
lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)
lineNumber = np.sort(lineNumber)[::-1]

for n in tqdm(range(nfolds)):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

for n in tqdm(range(nfolds)):
    values.append([lineNumber[n]])    

s=[]
for n in tqdm(range(nfolds)):
    s.append(mat[lineNumber[n],:])    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in tqdm(range(nind-1)):    
    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)
        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
        
        f = totDist[cycle]/np.sum(totDist[cycle])
        j = 0
        kn = 0
        
        for val in f:
            j += val        
            if (j > luck[cycle]):
                break
            kn +=1
        
        lineNumber[cycle] = kn
        
        for n_iter in range(nfolds):    
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)

for n_mod in tqdm(range(nfolds)):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
train_nn[['stock_id','time_id','target']] = train_df[['stock_id','time_id','target']]
test_nn[['stock_id','time_id']] = test_df[['stock_id','time_id']]
print(f"train_nn: {train_nn.shape} \ntest_nn: {test_nn.shape}")

del train_df
gc.collect()

In [ ]:
# Generate aggregate features
train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()
ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=0).fit(corr.values)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    
mat = []
matTest = []

n = 0
for ind in l:
    newDf = train_nn.loc[train_nn['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    mat.append ( newDf )
    
    newDf = test_nn.loc[test_nn['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(n)+'c1'
    matTest.append ( newDf )
    
    n+=1
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'], inplace=True)

mat2 = pd.concat(matTest).reset_index()
mat2 = pd.concat([mat2, mat1.loc[mat1.time_id==5]])

mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

In [ ]:
nnn = [
    'time_id',
    'trade_size_sum_0c1',
    'trade_size_sum_1c1',
    'trade_size_sum_3c1',
    'trade_size_sum_4c1',
    'trade_size_sum_6c1',
    'trade_order_count_sum_0c1',
    'trade_order_count_sum_1c1',
    'trade_order_count_sum_2c1',
    'trade_order_count_sum_4c1',
    'trade_order_count_sum_6c1',
    'trade_log_return_realized_volatility_0c1',
    'trade_log_return_realized_volatility_1c1',
    'trade_log_return_realized_volatility_3c1',
    'trade_log_return_realized_volatility_4c1',
    'trade_log_return_realized_volatility_6c1',
    'book_log_return1_realized_volatility_0c1',
    'book_log_return1_realized_volatility_1c1',
    'book_log_return1_realized_volatility_3c1',
    'book_log_return1_realized_volatility_4c1',
    'book_log_return1_realized_volatility_6c1',
    'book_bid_spread_sum_0c1',
    'book_bid_spread_sum_1c1',
    'book_bid_spread_sum_3c1',
    'book_bid_spread_sum_4c1',
    'book_bid_spread_sum_6c1',
    'book_ask_spread_sum_0c1',
    'book_ask_spread_sum_1c1',
    'book_ask_spread_sum_3c1',
    'book_ask_spread_sum_4c1',
    'book_ask_spread_sum_6c1',
    'book_total_volume_sum_0c1',
    'book_total_volume_sum_1c1',
    'book_total_volume_sum_3c1',
    'book_total_volume_sum_4c1',
    'book_total_volume_sum_6c1',
    'book_volume_imbalance_sum_0c1',
    'book_volume_imbalance_sum_1c1',
    'book_volume_imbalance_sum_3c1',
    'book_volume_imbalance_sum_4c1',
    'book_volume_imbalance_sum_6c1',
    'book_waph1_historical_volatility_0c1',
    'book_waph1_historical_volatility_1c1',
    'book_waph1_historical_volatility_3c1',
    'book_waph1_historical_volatility_4c1',
    'book_waph1_historical_volatility_6c1',
    'size_tau_0c1',
    'size_tau_1c1',
    'size_tau_3c1',
    'size_tau_4c1',
    'size_tau_6c1',
]

train_nn = pd.merge(train_nn, mat1[nnn], how='left', on='time_id')
test_nn = pd.merge(test_nn, mat2[nnn], how='left', on='time_id')
print(f"train_nn: {train_nn.shape} \ntest_nn: {test_nn.shape}")

del mat1,mat2
gc.collect()

In [ ]:
# Get highly correlated columns to remove
# df = train_nn.loc[:, train_nn.columns != 'target'].copy()
# cor_matrix = df.corr().abs()
# upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
# print(f"Number of columns to drop: {len(to_drop)}")
# select_cols = [col for col in test_nn.columns if col not in to_drop]
# print(f"Selected columns: {len(select_cols)}")
# print(select_cols)

In [ ]:
select_cols = [
    'trade_price_mean', 'trade_price_std', 'trade_price_sum', 'trade_size_mean', 'trade_size_std', 'trade_size_sum', 'trade_order_count_mean', 'trade_order_count_std',
    'trade_order_count_sum', 'trade_size_order_count_ratio_mean', 'trade_size_order_count_ratio_std', 'trade_log_return_realized_volatility', 'trade_log_return_mean',
    'trade_log_return_std', 'trade_log_return_sum', 'trade_price1_historical_volatility', 'trade_price1_sum', 'trade_order_count_std_120', 'trade_log_return_mean_120',
    'trade_log_return_sum_120', 'trade_price_std_240', 'trade_order_count_mean_240', 'trade_order_count_std_240', 'trade_log_return_mean_240', 
    'trade_log_return_sum_240', 'trade_price_std_360', 'trade_size_std_360', 'trade_order_count_mean_360', 'trade_order_count_std_360', 
    'trade_size_order_count_ratio_std_360', 'trade_log_return_mean_360', 'trade_log_return_sum_360', 'trade_price_std_480', 'trade_size_mean_480', 
    'trade_size_std_480', 'trade_size_sum_480', 'trade_order_count_mean_480', 'trade_order_count_std_480', 'trade_order_count_sum_480', 
    'trade_size_order_count_ratio_mean_480', 'trade_size_order_count_ratio_std_480', 'trade_log_return_realized_volatility_480', 'trade_log_return_mean_480', 
    'trade_log_return_std_480', 'trade_log_return_sum_480', 'book_wap1_sum', 'book_log_return1_realized_volatility', 'book_log_return1_mean', 'book_wap_balance_mean',
    'book_wap_balance_sum', 'book_price_spread1_std', 'book_price_spread2_mean', 'book_price_spread2_sum', 'book_bid_spread_mean', 'book_bid_spread_std',
    'book_bid_spread_sum', 'book_ask_spread_mean', 'book_ask_spread_std', 'book_ask_spread_sum', 'book_bid_ask_spread1_mean', 'book_bid_ask_spread1_std',
    'book_bid_ask_spread1_sum', 'book_bid_ask_spread2_mean', 'book_bid_ask_spread2_std', 'book_bid_ask_spread2_sum', 'book_total_volume_mean', 'book_total_volume_std',
    'book_log_return1_mean_120', 'book_log_return3_mean_120', 'book_log_return4_mean_120', 'book_log_return1_mean_240', 'book_log_return2_mean_240',
    'book_log_return3_mean_240', 'book_log_return4_mean_240', 'book_log_return1_mean_360', 'book_log_return2_mean_360', 'book_log_return3_mean_360',
    'book_log_return4_mean_360', 'book_waph1_std_360', 'book_log_return1_mean_480', 'book_log_return2_mean_480', 'book_log_return3_mean_480',
    'book_log_return4_mean_480', 'book_waph1_std_480', 'trade_log_return_realized_volatility_mean_stock', 'trade_log_return_realized_volatility_std_stock',
    'trade_log_return_realized_volatility_max_stock', 'trade_log_return_realized_volatility_min_stock', 'trade_log_return_realized_volatility_120_min_stock',
    'trade_log_return_realized_volatility_240_min_stock', 'trade_log_return_realized_volatility_360_max_stock', 'trade_log_return_realized_volatility_360_min_stock',
    'trade_log_return_realized_volatility_480_max_stock', 'trade_log_return_realized_volatility_480_min_stock', 'book_log_return1_realized_volatility_mean_stock',
    'book_log_return1_realized_volatility_std_stock', 'book_log_return1_realized_volatility_max_stock', 'book_log_return1_realized_volatility_min_stock', 
    'book_log_return2_realized_volatility_max_stock', 'book_log_return2_realized_volatility_min_stock', 'book_log_return3_realized_volatility_min_stock', 
    'book_log_return4_realized_volatility_max_stock', 'book_log_return4_realized_volatility_min_stock', 'book_log_return2_realized_volatility_120_min_stock',
    'book_log_return3_realized_volatility_120_max_stock', 'book_log_return3_realized_volatility_120_min_stock', 'book_log_return4_realized_volatility_120_max_stock',
    'book_log_return4_realized_volatility_120_min_stock', 'book_log_return1_realized_volatility_240_min_stock', 'book_log_return3_realized_volatility_240_max_stock',
    'book_log_return3_realized_volatility_240_min_stock', 'book_log_return4_realized_volatility_240_max_stock', 'book_log_return4_realized_volatility_240_min_stock',
    'book_log_return1_realized_volatility_360_min_stock', 'book_log_return2_realized_volatility_360_min_stock', 'book_log_return3_realized_volatility_360_max_stock',
    'book_log_return3_realized_volatility_360_min_stock', 'book_log_return4_realized_volatility_360_min_stock', 'book_log_return1_realized_volatility_480_max_stock',
    'book_log_return1_realized_volatility_480_min_stock', 'book_log_return2_realized_volatility_480_max_stock', 'book_log_return2_realized_volatility_480_min_stock', 
    'book_log_return3_realized_volatility_480_max_stock', 'book_log_return4_realized_volatility_480_max_stock', 'trade_log_return_realized_volatility_mean_time', 
    'trade_log_return_realized_volatility_std_time', 'trade_log_return_realized_volatility_max_time', 'trade_log_return_realized_volatility_min_time', 
    'trade_log_return_realized_volatility_120_min_time', 'trade_log_return_realized_volatility_240_min_time', 'trade_log_return_realized_volatility_360_min_time',
    'trade_log_return_realized_volatility_480_max_time', 'trade_log_return_realized_volatility_480_min_time', 'book_log_return1_realized_volatility_std_time',
    'book_log_return1_realized_volatility_max_time', 'book_log_return1_realized_volatility_min_time', 'book_log_return2_realized_volatility_max_time', 
    'book_log_return4_realized_volatility_min_time', 'book_log_return2_realized_volatility_360_min_time', 'book_log_return1_realized_volatility_480_max_time',
    'book_log_return1_realized_volatility_480_min_time', 'book_log_return2_realized_volatility_480_max_time', 'book_log_return2_realized_volatility_480_min_time',
    'book_log_return3_realized_volatility_480_max_time', 'book_log_return4_realized_volatility_480_max_time', 'trade_size_sum_0c1_x', 'trade_size_sum_1c1_x',
    'trade_size_sum_3c1_x', 'trade_size_sum_4c1_x', 'trade_size_sum_6c1_x', 'trade_order_count_sum_1c1_x', 'trade_order_count_sum_2c1_x',
    'trade_order_count_sum_4c1_x', 'trade_order_count_sum_6c1_x', 'trade_log_return_realized_volatility_1c1_x', 'book_bid_spread_sum_0c1_x',
    'book_bid_spread_sum_1c1_x', 'book_ask_spread_sum_1c1_x', 'book_total_volume_sum_0c1_x', 'book_total_volume_sum_1c1_x', 'book_total_volume_sum_3c1_x',
    'book_total_volume_sum_4c1_x', 'book_total_volume_sum_6c1_x', 'book_volume_imbalance_sum_0c1_x', 'book_volume_imbalance_sum_1c1_x',
    'book_volume_imbalance_sum_3c1_x', 'book_volume_imbalance_sum_4c1_x', 'book_volume_imbalance_sum_6c1_x', 'book_waph1_historical_volatility_0c1_x',
    'book_waph1_historical_volatility_1c1_x', 'book_waph1_historical_volatility_3c1_x', 'book_waph1_historical_volatility_4c1_x',
    'book_waph1_historical_volatility_6c1_x', 'size_tau_0c1_x', 'size_tau_1c1_x', 'size_tau_3c1_x', 'size_tau_4c1_x', 'size_tau_6c1_x', 'stock_id', 'time_id',
    'trade_size_sum_0c1_y', 'trade_size_sum_1c1_y', 'trade_size_sum_4c1_y', 'trade_size_sum_6c1_y', 'trade_order_count_sum_0c1_y',
    'trade_order_count_sum_1c1_y', 'trade_order_count_sum_4c1_y', 'book_total_volume_sum_0c1_y', 'book_total_volume_sum_1c1_y', 'book_total_volume_sum_3c1_y',
    'book_total_volume_sum_4c1_y', 'book_total_volume_sum_6c1_y', 'book_volume_imbalance_sum_0c1_y', 'book_volume_imbalance_sum_1c1_y',
    'book_volume_imbalance_sum_3c1_y', 'book_volume_imbalance_sum_4c1_y', 'book_volume_imbalance_sum_6c1_y', 'target'
]

len(select_cols)

In [ ]:
for col in tqdm([col for col in select_cols if col not in {"stock_id", "time_id", "target", "row_id"}]):
    qt = QuantileTransformer(random_state=21,
                             n_quantiles=2000, 
                             output_distribution='normal')
    
    train_nn[col] = qt.fit_transform(train_nn[[col]])
    test_nn[col] = qt.transform(test_nn[[col]])

In [ ]:
train_nn = train_nn[select_cols].copy()
test_nn = test_nn[[col for col in select_cols if col!='target']].copy()

features_to_consider = list(train_nn)
features_to_consider.remove('time_id')
features_to_consider.remove('target')

try:
    features_to_consider.remove('pred_NN')
except:
    pass

train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
print(f"train_nn: {train_nn.shape} \ntest_nn: {test_nn.shape}")

In [ ]:
cat_cols = ['stock_id']
num_cols = [col for col in test_nn.columns if not(col in cat_cols or col in {"time_id", "target", "row_id"})]
len(cat_cols), len(num_cols)

In [ ]:
def dnn_model(data, catcols, numcols):
    
    num_inp = Input(shape=(numcols,))
    
    inputs = []
    outputs = []
    
    for c in catcols:
        num_unique_values = int(data[c].max())
        embed_dim = 25
        inp = Input(shape=(1,))
        out = Embedding(input_dim=num_unique_values + 1, 
                        output_dim=embed_dim, 
                        embeddings_initializer='he_normal', 
                        name=c)(inp)
        out = SpatialDropout1D(rate=0.2)(out)
        out = Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    outputs.append(num_inp)
    x = Concatenate()(outputs)
    x = BatchNormalization()(x)
    
    x = Dense(units=512, kernel_initializer='he_normal', 
              kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.5)(x)
    
    x = Dense(units=128, kernel_initializer='he_normal', 
                kernel_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(rate=0.25)(x)

    x_output = Dense(units=1, kernel_initializer='he_normal')(x)
    
    model = Model(inputs=[inputs, num_inp], outputs=x_output, 
                  name='DNN_Model')
    return model

In [ ]:
model = dnn_model(train_nn, cat_cols, len(num_cols))
model.summary()

In [ ]:
plot_model(
    model, to_file='./Deep_Neural_Model.png', 
    show_shapes=True, show_layer_names=True
)

In [ ]:
target_name='target'
scores_folds = {}
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)
cat_data = train_nn['stock_id']
VERBOSE = 0

n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=2018)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train_nn)
features_to_consider.remove('time_id')
features_to_consider.remove('target')

try:
    features_to_consider.remove('pred_NN')
except:
    pass

train_nn[features_to_consider] = train_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())
test_nn[features_to_consider] = test_nn[features_to_consider].fillna(train_nn[features_to_consider].mean())

train_nn[pred_name] = 0
test_nn[target_name] = 0
test_predictions_nn = np.zeros(test_nn.shape[0])
y_pred_meta_dnn = np.zeros((train_nn.shape[0], 1))
y_pred_meta_dnn = pd.DataFrame(y_pred_meta_dnn, columns=['target'])

for n_count in range(n_folds):
    print(f'CV {counter}/{n_folds}')
    
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    
    X_train = train_nn.loc[train_nn.time_id.isin(indexes), features_to_consider]
    y_train = train_nn.loc[train_nn.time_id.isin(indexes), target_name]
    X_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), features_to_consider]
    y_test = train_nn.loc[train_nn.time_id.isin(values[n_count]), target_name]
    
    
    model = dnn_model(train_nn, cat_cols, len(num_cols))
    model.compile(loss=rmspe_loss, optimizer=Adamax(lr=1e-2))
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    num_data = X_train[features_to_consider]
    num_data = scaler.fit_transform(num_data.values)    
    
    cat_data = X_train['stock_id']    
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']
    
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20, verbose=VERBOSE,
        mode='min', restore_best_weights=True)

    plateau = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=7, 
        verbose=VERBOSE, mode='min')
    
    chkpoint = tf.keras.callbacks.ModelCheckpoint(
        f'./dnn_model_{counter}C.h5', 
        monitor='val_loss', verbose=VERBOSE, 
        save_best_only=True, mode='min')

    model.fit([cat_data, num_data], 
              target,               
              batch_size=2048,
              epochs=200,
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau, chkpoint],
              validation_batch_size=len(y_test),
              shuffle=True,
              verbose=VERBOSE)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    y_pred_meta_dnn.loc[train_nn.time_id.isin(values[n_count]), 'target'] = preds
    
    score = round(rmspe(y_true = y_test, y_pred = preds), 5)
    print(f'Fold: {counter} [{model_name}] - OOF RMSPE: {score}')
    scores_folds[model_name].append(score)
    
    tt = scaler.transform(test_nn[features_to_consider].values)
    test_predictions_nn += model.predict([test_nn['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10) / n_folds
       
    counter += 1
    features_to_consider.append('stock_id')

## Meta Model

* **Linear Regression**

In [ ]:
y_pred_final_lgb = np.array([y_pred_final_lgb]).T
y_pred_final_cb = np.array([y_pred_final_cb]).T

y_pred_final_lgb = y_pred_final_lgb.clip(0, 1e10)
y_pred_final_cb = y_pred_final_cb.clip(0, 1e10)

y_pred_final_dnn = np.array([test_predictions_nn.clip(0, 1e10)]).T
y_pred_meta_dnn = y_pred_meta_dnn.values.copy()

In [ ]:
Xtrain_meta = np.concatenate((y_pred_meta_lgb, y_pred_meta_cb, y_pred_meta_dnn), axis=1)
Xtest_meta = np.concatenate((y_pred_final_lgb, y_pred_final_cb, y_pred_final_dnn), axis=1)
Ytrain = train_nn['target'].copy()

print("Xtrain_meta shape: {}".format(Xtrain_meta.shape))
print("Xtest_meta shape: {}".format(Xtest_meta.shape))

In [ ]:
FOLD = 5
SEEDS = [2018]

oof_score = 0
y_pred_final = 0
counter = 0


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = KFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_nn)):
        counter += 1

        train_x, train_y = Xtrain_meta[train], Ytrain.iloc[train]
        val_x, val_y = Xtrain_meta[val], Ytrain.iloc[val]
        sample_weight = 1/np.square(train_y)

        model = LinearRegression()

        model.fit(train_x, train_y, sample_weight = sample_weight)

        y_pred = model.predict(val_x)
        y_pred_final += model.predict(Xtest_meta)
        
        score = rmspe(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("Seed-{} | Fold-{} | OOF Score: {}".format(seed, idx, score))
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_final = y_pred_final / float(counter)
oof_score /= float(counter)
print("Aggregate OOF Score: {}".format(oof_score))

## Create submission file

In [ ]:
submit_df = pd.DataFrame()
submit_df['row_id'] = test_df['row_id']
submit_df['target'] = y_pred_final
submit_df.to_csv("./submission.csv", index=False)
submit_df.head()